<a href="https://colab.research.google.com/github/guiOsorio/Learning_JAX/blob/master/MNIST_CometML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Comet Optimizer Tutorial - https://heartbeat.comet.ml/hyperparameter-optimization-with-comet-80c6d4b83502

In [1]:
# Install Comet
!pip install comet_ml --quiet

# Install Flax and JAX
!pip install --upgrade -q "jax[cuda11_cudnn805]" -f https://storage.googleapis.com/jax-releases/jax_releases.html
!pip install --upgrade -q git+https://github.com/google/flax.git

In [2]:
import jax
from jax import lax, random, jit, numpy as jnp

import flax
from flax.core import freeze, unfreeze
from flax import linen as nn
from flax.training import train_state

import optax

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

import functools
from typing import Sequence, Callable, Any, Optional

import numpy as np
import matplotlib.pyplot as plt

import comet_ml

In [3]:
def custom_transform(x):
    # A couple of modifications here compared to tutorial #3 since we're using a CNN
    # Input: (28, 28) uint8 [0, 255] torch.Tensor, Output: (28, 28, 1) float32 [0, 1] np array
    return np.expand_dims(np.array(x, dtype=np.float32), axis=2) / 255.

def custom_collate_fn(batch):
    """Provides us with batches of numpy arrays and not PyTorch's tensors."""
    transposed_data = list(zip(*batch))

    labels = np.array(transposed_data[1])
    imgs = np.stack(transposed_data[0])

    return imgs, labels

mnist_img_size = (28, 28, 1)
batch_size = 128

train_dataset = MNIST(root='train_mnist', train=True, download=True, transform=custom_transform)
test_dataset = MNIST(root='test_mnist', train=False, download=True, transform=custom_transform)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=custom_collate_fn, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, collate_fn=custom_collate_fn, drop_last=True)

# optimization - loading the whole dataset into memory
train_images = jnp.array(train_dataset.data)
train_lbls = jnp.array(train_dataset.targets)

# np.expand_dims is to convert shape from (10000, 28, 28) -> (10000, 28, 28, 1)
# We don't have to do this for training images because custom_transform does it for us.
test_images = np.expand_dims(jnp.array(test_dataset.data), axis=3)
test_lbls = jnp.array(test_dataset.targets)

print(train_images.shape, train_lbls.shape)
print('---')
print(test_images.shape, test_lbls.shape)
print('---')

for img, lbl in train_loader:
  print(img.shape)
  print(lbl.shape)
  break

(60000, 28, 28) (60000,)
---
(10000, 28, 28, 1) (10000,)
---
(128, 28, 28, 1)
(128,)


In [4]:
# 3. Implementation with batch norm, dropout and l2 regularization
class NN_regularized(nn.Module):
  dropout_rate: float

  @nn.compact 
  def __call__(self, x, train: bool):
    # Flatten x
    x = x.reshape((x.shape[0], -1))

    # Linear + dropout + relu
    x = nn.Dense(features=100)(x)
    x = nn.Dropout(self.dropout_rate, deterministic=not train)(x)
    x = nn.relu(x)

    # Linear + batch norm + relu
    x = nn.Dense(features=256)(x)
    x = nn.BatchNorm(use_running_average=not train)(x)
    x = nn.relu(x)

    # Linear + softmax
    x = nn.Dense(features=10)(x)
    x = nn.log_softmax(x)
    return x

In [5]:
# TRAINING

# Compute loss and update - this will be computed many times, so it's best to jit it
@jit
def training_state(state, imgs, gt_labels):

  def crossEntropy_loss(params, batch_stats):
    logits, updates = NN_regularized(dropout_rate=experiment.get_parameter('dropout_rate')).apply({'params': params, 'batch_stats': batch_stats}, imgs, train=True, rngs={'dropout': jax.random.PRNGKey(0)}, mutable=['batch_stats'])
    # logits is a vector of probabilities predicted by the model (the highest value in the vector is the prediction)
    one_hot_gt_labels = jax.nn.one_hot(gt_labels, num_classes=10) # one hot encoded vector of labels
    # logits.shape and one_hot_gt_labels shape is (batch_size, num_classes)
    loss = -jnp.mean(jnp.sum(logits * one_hot_gt_labels, axis=-1)) # axis=-1 means sum over rows

    # Add l2 regularization
    alpha = 0.1
    def l2_loss(weights, alpha):
      return alpha * (weights ** 2).mean()
    
    loss += sum(
        l2_loss(w, alpha)
        for w in jax.tree_util.tree_leaves(params)
      )

    return loss, (logits, updates)
  
  (loss, (logits, updates)), grads = jax.value_and_grad(crossEntropy_loss, argnums=0, has_aux=True)(state.params, state.batch_stats)
  state = state.apply_gradients(grads=grads) # update state params based on grads calculated
  state = state.replace(batch_stats=updates['batch_stats']) # update state batch_stats variables

  ## Accuracy
  accuracy = jnp.mean(jnp.argmax(logits, -1) == gt_labels)
  metrics = {
      'loss': loss,
      'accuracy': accuracy
  }

  return state, metrics

# One epoch - need to add metrics part
def train_one_epoch(state, dataloader):
  batch_metrics = []
  for cnt, (imgs, labels) in enumerate(dataloader):
    state, metrics = training_state(state, imgs, labels)
    batch_metrics.append(metrics)

  batch_metrics_np = jax.device_get(batch_metrics)  # pull from the accelerator onto host (CPU)
  epoch_metrics_np = {
      k: np.mean([metrics[k] for metrics in batch_metrics_np])
      for k in batch_metrics_np[0]
  }

  return state, epoch_metrics_np

def create_train_state(key, learning_rate, momentum):
  # Create model
  NN = NN_regularized(dropout_rate=experiment.get_parameter('dropout_rate'))
  # Initialize parameters
  variables = NN.init(key, jnp.ones([1, *mnist_img_size]), train=False)
  params = variables['params']
  batch_stats_v = variables['batch_stats']
  del variables

  class TrainState_stats(train_state.TrainState):
    batch_stats: Any

  state = TrainState_stats.create(
    apply_fn=NN.apply,
    params=params,
    batch_stats=batch_stats_v,
    tx=optax.sgd(learning_rate, momentum)
  )

  return state

In [6]:
# EVALUATION

# Run one evaluation on test set
@jit
def eval_step(state, imgs, gt_labels):
  logits = NN_regularized(dropout_rate=experiment.get_parameter('dropout_rate')).apply({'params': state.params, 'batch_stats': state.batch_stats}, imgs, rngs={'dropout': jax.random.PRNGKey(0)}, train=False)
  one_hot_gt_labels = jax.nn.one_hot(gt_labels, num_classes=10)
  loss = -jnp.mean(jnp.sum(logits * one_hot_gt_labels, axis=-1))
  accuracy = jnp.mean(jnp.argmax(logits, -1) == gt_labels)
  metrics = {
      'loss': loss,
      'accuracy': accuracy
  }
  return metrics

def evaluate_model(state, test_imgs, test_labels):
  metrics = eval_step(state, test_imgs, test_labels)
  metrics = jax.device_get(metrics) # pull from accelerator to CPU
  metrics = jax.tree_map(lambda x: x.item(), metrics) # get scalar value from array
  return metrics

In [7]:
# COMET OPTIMIZER CONFIG

spec = {
    'maxCombo': 20,
    'objective': 'minimize',
    'metric': 'loss',
    'minSampleSize': 500,
    'retryLimit': 20,
    'retryAssignLimit': 0
}

# model_params contains the info of the hyperparameters to optimize for - in this case learning_rate, momentum and dropout rate
model_params = {
    'learning_rate': {
        'type': 'float',
        'scaling_type': 'uniform',
        'min': 0.0001,
        'max': 0.4
    },
    'momentum': {
        'type': 'float',
        'scaling_type': 'uniform',
        'min': 0.5,
        'max': 0.99
    },
    'dropout_rate': {
        'type': 'float',
        'scaling_type': 'uniform',
        'min': 0.2,
        'max': 0.6
    }
}

config = {
    'algorithm': 'bayes',
    'spec': spec,
    'parameters': model_params,
    'name': 'Bayes MNIST optimizer',
    'trials': 1
}

In [8]:
opt = comet_ml.Optimizer(
    api_key = 'tSIIIrf40FyA0qDCJeLLC5jZP',
    config = config,
    project_name = 'mnist',
    workspace = 'guiosorio'
)

COMET WARNING: Passing Experiment through Optimizer constructor is deprecated; pass them to Optimizer.get_experiments or Optimizer.next
COMET INFO: COMET_OPTIMIZER_ID=e58c661320d243249391bf779ce1b77c
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 'infinite', 'endTime': None, 'id': 'e58c661320d243249391bf779ce1b77c', 'lastUpdateTime': None, 'maxCombo': 20, 'name': 'Bayes MNIST optimizer', 'parameters': {'dropout_rate': {'max': 0.6, 'min': 0.2, 'scalingType': 'uniform', 'scaling_type': 'uniform', 'type': 'float'}, 'learning_rate': {'max': 0.4, 'min': 0.0001, 'scalingType': 'uniform', 'scaling_type': 'uniform', 'type': 'float'}, 'momentum': {'max': 0.99, 'min': 0.5, 'scalingType': 'uniform', 'scaling_type': 'uniform', 'type': 'float'}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 20, 'metric': 'loss', 'minSampleSize': 500, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit': 20}, 'startTime': 59558621195, 'state': {'mode': None, 'seed': 

In [10]:
# FIT

# optimize lr, momentum, dropout hyperparameter with comet

for experiment in opt.get_experiments():

  from flax.training import train_state
  seed = 0
  learning_rate = experiment.get_parameter('learning_rate')
  momentum = experiment.get_parameter('momentum')
  n_epochs = 5

  train_state = create_train_state(jax.random.PRNGKey(seed), learning_rate, momentum)

  for epoch in range(n_epochs):
    print(f'EPOCH {epoch+1}')

    train_state, train_metrics = train_one_epoch(train_state, train_loader)
    print(f'Train accuracy: {train_metrics["accuracy"]}, Train loss: {train_metrics["loss"]}')
    experiment.log_metrics(train_metrics)

    test_metrics = evaluate_model(train_state, test_images, test_lbls)
    print(f'Test accuracy: {test_metrics["accuracy"]}, Test loss: {test_metrics["loss"]}')
    print(' ')
    experiment.log_metrics(test_metrics, prefix='test')

  experiment.end()

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET INFO: Optimizer metrics is 'loss' but no logged values found. Experiment ignored in sweep.
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/guiosorio/mnist/274bd37b07fc45a0a626f03df54e3760
COMET INFO:   Others:
COMET INFO:     optimizer_count        : 12
COMET INFO:     optimizer_id           : e58c661320d243249391bf779ce1b77c
COMET INFO:     optimizer_metric       : loss
COMET INFO:     optimizer_metric_value : 1
COMET INFO:     optimizer_name         : Bayes MNIST optimizer
COMET INFO:     optimizer_objective    : minimum
COMET INFO:     optimizer_parameters  

EPOCH 1
Train accuracy: 0.9086371660232544, Train loss: 0.37469592690467834
Test accuracy: 0.9362999796867371, Test loss: 26.12053108215332
 
EPOCH 2
Train accuracy: 0.9505876302719116, Train loss: 0.21365340054035187
Test accuracy: 0.9424999952316284, Test loss: 19.497312545776367
 
EPOCH 3
Train accuracy: 0.9594017267227173, Train loss: 0.16916216909885406
Test accuracy: 0.952299952507019, Test loss: 14.033671379089355
 
EPOCH 4
Train accuracy: 0.964476466178894, Train loss: 0.1437148004770279
Test accuracy: 0.9511999487876892, Test loss: 14.693293571472168
 
EPOCH 5
Train accuracy: 0.9671808481216431, Train loss: 0.13195067644119263
Test accuracy: 0.9402999877929688, Test loss: 17.772151947021484
 


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/guiosorio/mnist/2c20343326ce4c1ba7e9b5599a6c392b
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [5]      : (0.9086371660232544, 0.9671808481216431)
COMET INFO:     loss [5]          : (0.13195067644119263, 0.37469592690467834)
COMET INFO:     test_accuracy [5] : (0.9362999796867371, 0.952299952507019)
COMET INFO:     test_loss [5]     : (14.033671379089355, 26.12053108215332)
COMET INFO:   Others:
COMET INFO:     optimizer_count        : 13
COMET INFO:     optimizer_id           : e58c661320d243249391bf779ce1b77c
COMET INFO:     optimizer_metric       : loss
COMET INFO:     optimizer_metric_value : 0.13195067644119263
COMET INFO:     optimizer_name         : Bayes MNIST optimizer
COMET INFO:     optimizer_objective    : minimu

EPOCH 1
Train accuracy: 0.8100627660751343, Train loss: 0.9506143927574158
Test accuracy: 0.8715999722480774, Test loss: 115.80897521972656
 
EPOCH 2
Train accuracy: 0.8671541213989258, Train loss: 0.7475684881210327
Test accuracy: 0.7561999559402466, Test loss: 209.1077880859375
 
EPOCH 3
Train accuracy: 0.8881710767745972, Train loss: 0.6242477297782898
Test accuracy: 0.913599967956543, Test loss: 67.6955795288086
 
EPOCH 4
Train accuracy: 0.8996894955635071, Train loss: 0.5376530885696411
Test accuracy: 0.8819999694824219, Test loss: 87.09827423095703
 
EPOCH 5
Train accuracy: 0.9045973420143127, Train loss: 0.5094394087791443
Test accuracy: 0.9070000052452087, Test loss: 70.0921630859375
 


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/guiosorio/mnist/8c082b2475a24e658460f5c93b4b899d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [5]      : (0.8100627660751343, 0.9045973420143127)
COMET INFO:     loss [5]          : (0.5094394087791443, 0.9506143927574158)
COMET INFO:     test_accuracy [5] : (0.7561999559402466, 0.913599967956543)
COMET INFO:     test_loss [5]     : (67.6955795288086, 209.1077880859375)
COMET INFO:   Others:
COMET INFO:     optimizer_count        : 14
COMET INFO:     optimizer_id           : e58c661320d243249391bf779ce1b77c
COMET INFO:     optimizer_metric       : loss
COMET INFO:     optimizer_metric_value : 0.5094394087791443
COMET INFO:     optimizer_name         : Bayes MNIST optimizer
COMET INFO:     optimizer_objective    : minimum
COM

...

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/guiosorio/mnist/0dea7ffe4f054b46bfe4fe020c66a6ef
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [5]      : (0.8970686197280884, 0.9643930196762085)
COMET INFO:     loss [5]          : (0.1613520085811615, 0.42787235975265503)
COMET INFO:     test_accuracy [5] : (0.9469999670982361, 0.9535999894142151)
COMET INFO:     test_loss [5]     : (15.796197891235352, 20.257667541503906)
COMET INFO:   Others:
COMET INFO:     optimizer_count        : 18
COMET INFO:     optimizer_id           : e58c661320d243249391bf779ce1b77c
COMET INFO:     optimizer_metric       : loss
COMET INFO:     optimizer_metric_value : 0.1613520085811615
COMET INFO:     optimizer_name         : Bayes MNIST optimizer
COMET INFO:     optimizer_objective    : minimu

EPOCH 1
Train accuracy: 0.8904580473899841, Train loss: 0.43000128865242004
Test accuracy: 0.9425999522209167, Test loss: 20.344202041625977
 
EPOCH 2
Train accuracy: 0.9377837777137756, Train loss: 0.25048476457595825
Test accuracy: 0.9542999863624573, Test loss: 13.474527359008789
 
EPOCH 3
Train accuracy: 0.9461471438407898, Train loss: 0.20689500868320465
Test accuracy: 0.9508000016212463, Test loss: 14.134186744689941
 
EPOCH 4
Train accuracy: 0.9513221383094788, Train loss: 0.18288736045360565
Test accuracy: 0.9490999579429626, Test loss: 14.279651641845703
 
EPOCH 5
Train accuracy: 0.9569144248962402, Train loss: 0.16662593185901642
Test accuracy: 0.9492999911308289, Test loss: 14.599078178405762
 


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/guiosorio/mnist/47e4b35d79e1434b91347a8c132c9e94
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [5]      : (0.8904580473899841, 0.9569144248962402)
COMET INFO:     loss [5]          : (0.16662593185901642, 0.43000128865242004)
COMET INFO:     test_accuracy [5] : (0.9425999522209167, 0.9542999863624573)
COMET INFO:     test_loss [5]     : (13.474527359008789, 20.344202041625977)
COMET INFO:   Others:
COMET INFO:     optimizer_count        : 19
COMET INFO:     optimizer_id           : e58c661320d243249391bf779ce1b77c
COMET INFO:     optimizer_metric       : loss
COMET INFO:     optimizer_metric_value : 0.16662593185901642
COMET INFO:     optimizer_name         : Bayes MNIST optimizer
COMET INFO:     optimizer_objective    : mini

EPOCH 1
Train accuracy: 0.8674378991127014, Train loss: 0.5063132047653198
Test accuracy: 0.9304999709129333, Test loss: 25.459430694580078
 
EPOCH 2
Train accuracy: 0.9220085740089417, Train loss: 0.30876827239990234
Test accuracy: 0.9548999667167664, Test loss: 12.646886825561523
 
EPOCH 3
Train accuracy: 0.9343449473381042, Train loss: 0.25399890542030334
Test accuracy: 0.9523999691009521, Test loss: 13.895369529724121
 
EPOCH 4
Train accuracy: 0.939636766910553, Train loss: 0.22992460429668427
Test accuracy: 0.952299952507019, Test loss: 13.365518569946289
 
EPOCH 5
Train accuracy: 0.9452958106994629, Train loss: 0.20846767723560333
Test accuracy: 0.9603999853134155, Test loss: 10.276925086975098
 


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/guiosorio/mnist/4bb961294092482e87e1f199fc16fb5a
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [5]      : (0.8674378991127014, 0.9452958106994629)
COMET INFO:     loss [5]          : (0.20846767723560333, 0.5063132047653198)
COMET INFO:     test_accuracy [5] : (0.9304999709129333, 0.9603999853134155)
COMET INFO:     test_loss [5]     : (10.276925086975098, 25.459430694580078)
COMET INFO:   Others:
COMET INFO:     optimizer_count        : 20
COMET INFO:     optimizer_id           : e58c661320d243249391bf779ce1b77c
COMET INFO:     optimizer_metric       : loss
COMET INFO:     optimizer_metric_value : 0.20846767723560333
COMET INFO:     optimizer_name         : Bayes MNIST optimizer
COMET INFO:     optimizer_objective    : minim